<a href="https://colab.research.google.com/github/evtimovr/Bla/blob/master/Finassign7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from PIL import Image # library 'pillow' for image processing
import os, os.path
import glob
from os.path import basename
import re
regex=r'[a-zA-Z_]+(?=_\d)'

image_list = []
image_label=[]
for filename in glob.glob('/content/drive/My Drive/Final assignment/training/*/*.jpg'): #assuming jpg
    im=Image.open(filename)
    name=re.findall(regex,filename)
    image_label.append(name)
    image_list.append(im)

KeyboardInterrupt: ignored

In [0]:
from PIL import Image # library 'pillow' for image processing
import os, os.path
import glob
from os.path import basename
import re

In [0]:
image_label[100]

NameError: ignored

In [0]:

    
for n, i in enumerate(image_label):
     if i == ['easy']:
         image_label[n] = ['fake']
        
for n, i in enumerate(image_label):
     if i == ['hard']:
         image_label[n] = ['fake']
        
for n, i in enumerate(image_label):
     if i == ['mid']:
         image_label[n] = ['fake']

In [0]:
len(image_list)

1354

!!!Show you know what the sizes are

In [0]:
print(sum(x == ['fake'] for x in image_label))
print(sum(x == ['real'] for x in image_label))



463
891


In [0]:
from keras.preprocessing.image import ImageDataGenerator 
sqrt_dim=40
train_batch_size=20
input_shape=(sqrt_dim, sqrt_dim, 3)

train_generator = ImageDataGenerator(rescale = 1./255, 
                                   shear_range = 0.1, 
                                   zoom_range = 0.1,
                                   horizontal_flip = False) 

In [23]:

test_generator = ImageDataGenerator(rescale = 1./255) # only rescaling with fixed parameters is valid, otherwise do not touch the test set! 

trainset = train_generator.flow_from_directory('/content/drive/My Drive/Final assignment/train', # specify the directory (you can right click on the folder you need and copy the path)
                                                 target_size = (sqrt_dim, sqrt_dim), # you need to check your data,
                                                 batch_size = train_batch_size, # batch size = 5 here just for show!
                                                 class_mode= 'categorical')





Found 725 images belonging to 2 classes.


In [24]:
validationset = test_generator.flow_from_directory('/content/drive/My Drive/Final assignment/validation', # specify the directory (you can right click on the folder you need and copy the path)
                                                 target_size = (sqrt_dim, sqrt_dim), # you need to check your data,
                                                 batch_size = train_batch_size, # batch size = 5 here just for show!
                                                 class_mode= 'categorical')

Found 200 images belonging to 2 classes.


In [25]:
trainset.class_indices

{'fake': 0, 'real': 1}

In [26]:
n_train_samples = len(trainset.filenames)
n_classes = len(trainset.class_indices)
print(n_train_samples, n_classes)

725 2


**CNN Start**

In [0]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Dropout # converting pooled feature maps into a large feature vector to prepare for input into the Dense layer
from keras.layers import Conv2D # first convolution step, 3D for videos
from keras.layers import MaxPooling2D #pooling, downsampling

model = Sequential() #initialize
model.add(Conv2D(64, #number of filters/feature detectors, number of feature maps you will end up with. Starting with 32 is good practice, grow it if needed and if you have enough computing power
                 kernel_size=(3, 3), # size of filters (rows,columns), (3,3) is also a popular choice, it's a hyperparameter to choose
                 activation='relu', # removes the negative pixels, introducing non-linearity, 
                 #we are skipping "border mode" - it is set to "same" by default
                 # strides=(2,2) how the window shifts by in each of the dimensions. Default stride is 1 but we will go for 2 for faster implementation by reducing the image size similar to pooling
                 input_shape=input_shape)) # (3 if color/1 if b&w, height, width), all images have to come in a single format
model.add(MaxPooling2D(pool_size=(2, 2), #the size of the pooled feature map
                       strides=None)) # default: pool_size, here: (2,2)
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(128, (3, 3), activation='relu',padding = 'same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))


model.add(Conv2D(16, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu')) # that's our classifier bit, th size is up to you, choose wisely taking the size of picture in mind (smth between nuber of input and number of output nodes could be another rule of thumb)
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='softmax')) 

In [42]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           (None, 38, 38, 64)        1792      
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 19, 19, 64)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 17, 17, 128)       73856     
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 8, 8, 128)         0         
_________________________________________________________________
dropout_26 (Dropout)         (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 8, 8, 128)         147584    
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 4, 4, 128)         0         
__________

In [0]:
max_epochs = 40

In [43]:
import tensorflow as tf
class AccuracyCallback(tf.keras.callbacks.Callback):
    """Callback that stops training on requested accuracy."""
    def __init__(self, target_accuracy=0.997):
        self.target_accuracy = target_accuracy
    def on_epoch_end(self, epoch, logs):
        if logs.get('val_acc') >= self.target_accuracy:
            print("Target validation accuracy (%f) reached !" % self.target_accuracy )
            self.model.stop_training = True
            
target_accuracy = 0.62
max_epochs = 40
print("\nTraining untill accuracy reaches %s or for %i epochs" %(target_accuracy, max_epochs))


Training untill accuracy reaches 0.62 or for 40 epochs


In [44]:
history=model.fit_generator(trainset,
                    steps_per_epoch= 400,# number of times generator will be used,that's how we control for the number of generated images, overall size of set divided by the batch size to make sure everything gets used
                    validation_data=validationset,
                    validation_steps= 10, # 340/2
                    epochs=25,
                    callbacks=[AccuracyCallback(target_accuracy)]
)

Epoch 1/25
400/400 [==============================] - 107s 269ms/step - loss: 0.6960 - acc: 0.4940 - val_loss: 0.6929 - val_acc: 0.5000
Epoch 2/25
400/400 [==============================] - 107s 267ms/step - loss: 0.6934 - acc: 0.5098 - val_loss: 0.6913 - val_acc: 0.5550
Epoch 3/25
400/400 [==============================] - 108s 269ms/step - loss: 0.6941 - acc: 0.4898 - val_loss: 0.6922 - val_acc: 0.5150
Epoch 4/25
400/400 [==============================] - 107s 269ms/step - loss: 0.6934 - acc: 0.5136 - val_loss: 0.6934 - val_acc: 0.4800
Epoch 5/25
400/400 [==============================] - 108s 269ms/step - loss: 0.6931 - acc: 0.5022 - val_loss: 0.6888 - val_acc: 0.5400
Epoch 6/25
400/400 [==============================] - 107s 268ms/step - loss: 0.6932 - acc: 0.5148 - val_loss: 0.6894 - val_acc: 0.5400
Epoch 7/25
400/400 [==============================] - 107s 268ms/step - loss: 0.6869 - acc: 0.5452 - val_loss: 0.6920 - val_acc: 0.4800
Epoch 8/25
400/400 [============================

In [0]:
model.save('model7')


In [0]:
unknownset = test_generator.flow_from_directory('/content/drive/My Drive/Final assignment/unknown', # specify the directory (you can right click on the folder you need and copy the path)
                                                 target_size = (sqrt_dim, sqrt_dim), # you need to check your data,
                                                 batch_size = train_batch_size, # batch size = 5 here just for show!
                                                 class_mode= 'categorical')

Found 481 images belonging to 1 classes.


In [0]:
img = image.load_img('images/*', target_size=(40, 40))

NameError: ignored

In [0]:
from skimage.io import imread_collection

#your path 
col_dir = '/content/drive/My Drive/Final assignment/unknown/unknown/*'

#creating a collection with the available images
col = imread_collection(col_dir)


In [0]:
#You have to reset or reassign the test generator
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory('/content/drive/My Drive/Final assignment/unknown/',  # specify the directory
                                            target_size = (sqrt_dim,sqrt_dim),
                                            batch_size = 1, #to sample an image just once
                                            class_mode = 'categorical',
                                            shuffle = False)
filenames = test_generator.filenames
nb_samples = len(filenames)

yhat = model.predict_generator(test_generator,steps = nb_samples)

Found 481 images belonging to 1 classes.


In [0]:
filenames[1]

'unknown/003358d7a7461d8cdbe9.jpg'

In [0]:
import pandas as pd

names = []
predictions=[]

for i in range(0,len(filenames)):
     names.append(filenames[i])

for i in range(0,len(filenames)):
     predictions.append(yhat[i,0])
    
pred = pd.DataFrame({'ID':names,'fake':predictions})

In [0]:
pred.to_csv("predictions5.csv",header = True,index=False)

In [0]:
pred

,col
0,unknown/00217efccd6f697eb937.jpg
1,unknown/003358d7a7461d8cdbe9.jpg
2,unknown/008cae7827beecd1aab3.jpg
3,unknown/00b75601a2272c9c8f3e.jpg
4,unknown/01224275d3e25f62a920.jpg
5,unknown/01a1a6b856bffedc304c.jpg
6,unknown/020c00becfcd4e7938af.jpg
7,unknown/045bcecab3b5f0f68836.jpg
8,unknown/046d3ccaf5e4888cff8f.jpg
9,unknown/056907b340072139457a.jpg


In [0]:
model.predict(col[1])

ValueError: ignored